In [6]:
import requests
from requests.auth import HTTPBasicAuth
import json
import numpy as np
import pandas as pd
import ipywidgets as widgets
global token

auth_label = widgets.Label(value='NGL authentication (expires after 2 hours)')
username = widgets.Text(value='',placeholder='username',description='username',disabled=False)
password = widgets.Password(value='',placeholder='Enter password', description='password', disabled=False)
button = widgets.Button(description='Login',disabled=False)
out = widgets.Output()
auth_widget = widgets.VBox([username, password, button])
auth_panel = widgets.VBox([auth_label, auth_widget, out])
display(auth_panel)

def ngl_auth(b):
    global token
    url = 'https://nextgenerationliquefaction.org/api/query.php/auth'
    headers = {"User-Agent":"XY"}
    r = requests.get(url, headers=headers, auth=HTTPBasicAuth(username.value,password.value))
    if(not json.loads(r.text)['success']):
        with out:
            print(json.loads(r.text)['message'])
        return
    else:
        with out:
            print(json.loads(r.text)['message'])
        token = json.loads(r.text)['token']
        return token
    
def ngl_query(sql, token):
    url = 'https://nextgenerationliquefaction.org/api/query.php/query'
    headers = {"User-Agent":"XY", "Authorization": "Bearer {}".format(token)}
    data = {'sql':sql}
    r= requests.post(url, headers=headers, json=data)
    try:
        df = pd.DataFrame.from_dict(json.loads(r.text)['data'], orient='columns')
        df.fillna(value=np.nan, inplace=True)
    except:
        df = pd.DataFrame()
    
    for col in df.columns:
        try:
            s = df[col]
            df[col] = [int(i) if i.isdigit() else float(i) for i in s]
        except:
            continue
    
    for col in df.columns:
        if df[col].dtype != 'int64':
            try:
                df[col] = df[col].astype(float)
            except:
                continue
    return(df)

button.on_click(ngl_auth)

In [7]:
import numpy as np
sql = 'SELECT * FROM SITE'
df = ngl_query(sql, token)
df

,SITE_ID,SITE_NAME,SITE_LAT,SITE_LON,SITE_GEOL,SITE_REM,SITE_STAT,SITE_REVW
0,147,Amagasaki,34.715560,135.400750,Qal,Industrial site near Yomoga River. Coordinate...,1,2
1,148,Bonds Corner,32.693100,-115.338200,"Qal, deep, Imperial Valley",,1,2
2,149,Hachirogata,39.850000,140.017000,Af (Fill),Gingery indicates the geology near the strong ...,1,2
3,150,Higashi-Kobe Bridge,34.710214,135.293345,Af (Fill),A geologic map of the area (Geological Survey ...,1,2
4,151,Hanshin Expressway,34.724834,135.301489,,Hanshin Expressway (Mylonakis et al. 2006),1,2
...,...,...,...,...,...,...,...,...
418,847,Wufeng MAA BH-9,24.053842,120.693991,,Site data from PEER (2000): https://apps.peer....,0,0
419,848,"Wufeng MAA C-10, BH-10",24.053510,120.696770,,Site data from PEER (2000): https://apps.peer....,0,0
420,849,Wufeng MAA BH-11,24.062664,120.696644,,Site data from PEER (2000): https://apps.peer....,0,0
421,850,Wufeng MAA BH-12,24.050435,120.695026,,Site data from PEER (2000): https://apps.peer....,0,0


In [9]:
import numpy as np
sql = 'SELECT DISTINCT SCPG_ID FROM SCPG'
df = ngl_query(sql, token)
df

,SCPG_ID
0,254
1,1167
2,1081
3,1226
4,1154
...,...
934,1920
935,1921
936,1922
937,1923


In [ ]:
import requests
url = 'https://www.uclageo.com/gm_database/api/index.php/flatfile?format=json&tables=fourier_spectra,motion,time_series,basin_model&limit=1'
headers = {"User-Agent":"XY"}
r = requests.get(url,headers=headers)
print(json.loads(r.text)[0])